In [65]:
from datetime import datetime
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from tqdm import tqdm

from OddsAPI import API
api = API()
api.setAPI("RBmBCJ2TXHg57CyueRIo5YTrgYQLDgKhflMtQE")

In [88]:
LEAUGE_TYPE = "NBA"
BOOK = "underdog"

In [113]:
pd.set_option('display.max_rows', None)

In [109]:
## pivot: converts a dataframe that contains two seperate rows for each player (one for over, one for under) to have one row for each player
def pivot(df):
    df = df.copy()
    pivot_df = pd.pivot_table(df, values='odds', index=['handicap', 'participant_name'], columns='name').reset_index()
    try:
        pivot_df.columns = ['line', 'participant_name', 'over_odds', 'under_odds']
    except Exception as e:
        print(e)
        print(df)
        print(pivot_df)
        print(pivot_df.columns)
    return pivot_df

## calculate odds
## adapted from Ammar Sulmanjee
def calculate_odds(x, y=None):
    if not y:
        if x>= 0: return 1 + x/100
        return 1 + 100/abs(x)

    imp_prob1 = (1 / calculate_odds(x)) * 100
    imp_prob2 = (1 / calculate_odds(y)) * 100

    total_implied_prob = round(imp_prob1 + imp_prob2, 4)
    fair_prob1 = round(imp_prob1 / total_implied_prob * 100, 2)
    fair_prob2 = round(imp_prob2 / total_implied_prob * 100, 2)

    return [fair_prob1, fair_prob2]

## saves best bets
def download_bets(best_bets):
    best_bets['market'] = best_bets['market'].str.replace(r'_|player|over_under', '', regex=True).str.strip()
    best_bets = best_bets.drop(columns=['game'])
    best_bets.to_csv(f"Bets_{datetime.now().strftime('%Y-%m-%d')}.csv", index=None)

## check palyer's raw data (debugging function)
def check_player(raw_bet_data, name, market, line=None):
    pd.set_option('display.max_rows', None)
    if line:
        display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market) & (raw_bet_data['handicap'] == line)])
        pd.reset_option('display.max_rows')
        return
    display(raw_bet_data[(raw_bet_data['participant_name'] == name) & (raw_bet_data['market'] == market)])
    pd.reset_option('display.max_rows')


In [ ]:
## https://api.underdogfantasy.com/beta/v5/over_under_lines

In [89]:
### Pull Underdog Lines
# result: ud (DataFrame) contains underdog lines + names, games_to_check (dictionary) contains each game, with its corresponding markets to check

ud = pd.DataFrame.from_dict(api.get_fantasy_lines(LEAUGE_TYPE)['fantasy_books'])

ud = ud[ud['bookie_key'] == BOOK]
ud = pd.DataFrame.from_dict(list(ud["markets"])[0])
ud.columns = ['markets', 'lines']

df = pd.DataFrame(columns=['participant_name', 'line', 'game_id', 'market'])

lines = list(ud['lines'])
markets = list(ud['markets'])

for x, line in enumerate(lines):
    for prop in line:
        temp = pd.DataFrame()

        temp['participant_name'] = [prop['participant_name']]
        temp['line'] = [prop['line']]
        temp['game_id'] = [prop['game_id']]
        temp['market'] = markets[x]

        df = pd.concat([df, temp], axis=0)

df = df.reset_index()
df = df.drop(columns=['index'])
df = df.sort_values(by=['game_id'])


ud = df[['participant_name', 'line', 'market']]
ud.columns = [ud.columns[0], 'handicap', 'market']

result_df = df.groupby('game_id')['market'].agg(set).reset_index()
games_to_check = dict(zip(result_df['game_id'], result_df['market']))

In [90]:
bets = pd.DataFrame(columns = ['participant_name', 'market', 'line', 'over_odds', 'under_odds'])
raw_bet_data = pd.DataFrame()

for game, markets in tqdm(games_to_check.items(), desc="Processing games"):
    for market in markets:
            
        # get sportsbook odds
        try:
            bookies_data = api.get_most_recent_odds(game, market)
        except Exception as e:
            if str(e) == "ERROR 422": continue
            else: 
                print(e)
                continue
        books = []
        
        for i in range(len(bookies_data['sportsbooks'])):
            df = pd.DataFrame.from_dict(bookies_data['sportsbooks'][i]['market']['outcomes'])
            df = df[df['timestamp'].str.contains(datetime.now().strftime('%Y-%m-%d'))]

            df['book'] = bookies_data['sportsbooks'][i]['bookie_key']
            df['market'] = market

            books.append(df)

        raw_data = pd.concat(books)
        raw_bet_data = pd.concat([raw_bet_data, raw_data])

        # concat the books into one
        new_datasets = []
        for dataset in books:
            dataset = pd.merge(dataset[['handicap', 'odds', 'participant_name', 'name', 'market']], ud, how='inner', on = ['participant_name', 'handicap' , 'market'])
            dataset = dataset[['handicap', 'odds', 'participant_name', 'name']]
            
            if dataset.shape[0] > 0 and dataset.shape[1] > 0 and dataset.shape[0]%2 == 0:
                dataset = pivot(dataset)

                dataset = dataset[abs(dataset['over_odds']) < 225]
                dataset = dataset[abs(dataset['under_odds']) < 225]
                
                if dataset.shape[0] > 0 and dataset.shape[1] > 0:
                    new_datasets.append(dataset)
        if new_datasets == []: continue

        
        final = pd.concat(new_datasets, axis=0)
        final = final.dropna()

        final[['over_odds', 'under_odds']] = final.apply(lambda row: pd.Series(calculate_odds(row['over_odds'], row['under_odds'])), axis=1)
        final = final.groupby(['participant_name', 'line']).agg({'over_odds':'median', 'under_odds':'median'}).reset_index()


        final['ev'] = abs(final['over_odds'] - 50)
        final['market'] = market
        final['game'] = game


        if final.shape[0] > 0: 
            bets = pd.concat([bets, final[['participant_name', 'ev', 'market', 'line', 'over_odds', 'under_odds', 'game']]])

Processing games: 100%|██████████| 8/8 [00:44<00:00,  5.57s/it]


In [95]:
bets = bets.sort_values(by=['ev'], ascending=False)
bets = bets.dropna()
bets = bets.drop_duplicates(keep='first', ignore_index=True)

best_bets = bets.copy()
best_bets = best_bets[best_bets['ev'] > 4.5]

print(f"Total Bets found!: {bets.shape[0]}")
print(f"Total Positive EV Bets found!: {best_bets.shape[0]}")

Total Bets found!: 393
Total Positive EV Bets found!: 5


In [114]:
bets

,participant_name,market,line,over_odds,under_odds,ev,game
0,Jeremy Sochan,player_turnovers_over_under,1.5,55.975,44.025,5.975,b845c095fb257ca1620b867c4d9de530
1,Julian Champagnie,player_turnovers_over_under,0.5,55.88,44.12,5.880,b845c095fb257ca1620b867c4d9de530
2,Victor Wembanyama,player_turnovers_over_under,3.5,55.83,44.17,5.830,b845c095fb257ca1620b867c4d9de530
3,Jonathan Kuminga,player_steals_over_under,0.5,55.06,44.94,5.060,7aab939af74de8c8d7726c83a47ed6a3
4,Jonathan Kuminga,player_blocks_over_under,0.5,55.06,44.94,5.060,7aab939af74de8c8d7726c83a47ed6a3
5,Ayo Dosunmu,player_points_over_under,12.5,54.08,45.92,4.080,36e3f4950e19fa8511ad6030574d20db
6,Franz Wagner,player_threes_over_under,1.5,54.02,45.98,4.020,b845c095fb257ca1620b867c4d9de530
7,Domantas Sabonis,player_steals_over_under,0.5,54.02,45.98,4.020,c96521dde3008893e0f55e16327596c1
8,Markelle Fultz,player_assists_rebounds_over_under,6.5,53.62,46.38,3.620,b845c095fb257ca1620b867c4d9de530
9,Domantas Sabonis,player_rebounds_over_under,12.5,53.59,46.41,3.590,c96521dde3008893e0f55e16327596c1


In [106]:
display(best_bets)

,participant_name,market,line,over_odds,under_odds,ev,game
0,Jeremy Sochan,player_turnovers_over_under,1.5,55.975,44.025,5.975,b845c095fb257ca1620b867c4d9de530
1,Julian Champagnie,player_turnovers_over_under,0.5,55.88,44.12,5.880,b845c095fb257ca1620b867c4d9de530
2,Victor Wembanyama,player_turnovers_over_under,3.5,55.83,44.17,5.830,b845c095fb257ca1620b867c4d9de530
3,Jonathan Kuminga,player_steals_over_under,0.5,55.06,44.94,5.060,7aab939af74de8c8d7726c83a47ed6a3
4,Jonathan Kuminga,player_blocks_over_under,0.5,55.06,44.94,5.060,7aab939af74de8c8d7726c83a47ed6a3


In [98]:
ud[(ud['participant_name'] == 'Jeremy Sochan') & (ud['market'].str.contains('turnovers'))]

,participant_name,handicap,market
431,Jeremy Sochan,1.5,player_turnovers_over_under


In [105]:
a = pd.DataFrame.from_dict(api.get_fantasy_lines(LEAUGE_TYPE)['fantasy_books'])

a = a[a['bookie_key'] == BOOK]
a = pd.DataFrame.from_dict(list(a["markets"])[0])
a.columns = ['markets', 'lines']
a = a[a['markets'].str.contains('player_turnovers_over_under')]
list(lines)[0]

[{'timestamp': '2024-01-30T19:35:20.883631',
  'game_id': '36e3f4950e19fa8511ad6030574d20db',
  'line': 6.0,
  'participant': 365,
  'participant_name': 'Dennis Schroder'},
 {'timestamp': '2024-01-30T21:00:28.908649',
  'game_id': '8ef0604b7dedc854a51ff1410e7c6610',
  'line': 2.5,
  'participant': 366,
  'participant_name': 'Jordan Poole'},
 {'timestamp': '2024-01-17T04:00:06.544415',
  'game_id': '6a84df3782bc046cf5bda47fc177862d',
  'line': 2.0,
  'participant': 369,
  'participant_name': 'Kevon Looney'},
 {'timestamp': '2024-01-30T22:30:22.808608',
  'game_id': 'e9f33a7f036c43a32d2db5426f9bcc48',
  'line': 5.0,
  'participant': 377,
  'participant_name': 'Kevin Durant'},
 {'timestamp': '2024-01-30T06:10:22.257330',
  'game_id': '7aab939af74de8c8d7726c83a47ed6a3',
  'line': 1.5,
  'participant': 384,
  'participant_name': 'Andrew Wiggins'},
 {'timestamp': '2024-01-30T22:30:22.808894',
  'game_id': 'e9f33a7f036c43a32d2db5426f9bcc48',
  'line': 6.0,
  'participant': 386,
  'participant

In [96]:
display(best_bets)


,participant_name,market,line,over_odds,under_odds,ev,game
0,Jeremy Sochan,player_turnovers_over_under,1.5,55.975,44.025,5.975,b845c095fb257ca1620b867c4d9de530
1,Julian Champagnie,player_turnovers_over_under,0.5,55.88,44.12,5.880,b845c095fb257ca1620b867c4d9de530
2,Victor Wembanyama,player_turnovers_over_under,3.5,55.83,44.17,5.830,b845c095fb257ca1620b867c4d9de530
3,Jonathan Kuminga,player_steals_over_under,0.5,55.06,44.94,5.060,7aab939af74de8c8d7726c83a47ed6a3
4,Jonathan Kuminga,player_blocks_over_under,0.5,55.06,44.94,5.060,7aab939af74de8c8d7726c83a47ed6a3


In [93]:
download_bets(best_bets)

In [37]:
check_player(raw_bet_data, 'Alex Caruso', 'player_points_over_under', 12.5)

,timestamp,handicap,odds,participant,participant_name,name,description,book,market
2,2024-01-30T22:52:02,12.5,-102,17168.0,Alex Caruso,Over,Alex Caruso Over 12.5 Alex Caruso - Alt Points,fanduel,player_points_over_under
10,2024-01-30T22:44:02,12.5,104,17168.0,Alex Caruso,Over,Alex Caruso Over Alex Caruso - Points,fanduel,player_points_over_under
128,2024-01-30T22:52:02,12.5,-130,17168.0,Alex Caruso,Under,Alex Caruso Under 12.5 Alex Caruso - Alt Points,fanduel,player_points_over_under
132,2024-01-30T22:44:02,12.5,-128,17168.0,Alex Caruso,Under,Alex Caruso Under Alex Caruso - Points,fanduel,player_points_over_under
0,2024-01-30T16:12:06,12.5,105,17168.0,Alex Caruso,Over,Over Alex Caruso (Points),pinnacle,player_points_over_under
10,2024-01-30T21:36:02,12.5,-137,17168.0,Alex Caruso,Under,Under Alex Caruso (Points),pinnacle,player_points_over_under
2,2024-01-30T03:48:05,12.5,-110,17168.0,Alex Caruso,Over,Over - Alex Caruso Points,draftkings,player_points_over_under
35,2024-01-30T03:48:05,12.5,-120,17168.0,Alex Caruso,Under,Under - Alex Caruso Points,draftkings,player_points_over_under
0,2024-01-30T02:08:13,12.5,-113,17168.0,Alex Caruso,Over,Over Alex Caruso Total Points,caesars,player_points_over_under
10,2024-01-30T02:08:13,12.5,-121,17168.0,Alex Caruso,Under,Under Alex Caruso Total Points,caesars,player_points_over_under
